In [1]:
from utils import get_raw_data, dump_prediction
import fasttext
train_data, test_data, y_train, G, train_hosts, tests_hosts = \
    get_raw_data(encoding='utf-8', errors="ignore")

100%|█████████████████████████████████████████████████████████████████████████████████| 560/560 [00:00<00:00, 561110.90it/s]


In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
lab = LabelEncoder()
onhot = OneHotEncoder()
y_train_index = lab.fit_transform(y_train)
y_train_one_hot = onhot.fit_transform(y_train_index[:,np.newaxis])

In [3]:
from sklearn.model_selection import train_test_split
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(
    train_data, y_train, test_size=0.15, shuffle= True, stratify=y_train_index)

In [4]:
num_classes = len(set(y_train_index))

In [5]:
clean_tokenize = lambda sentence: re.sub(r'\W+', ' ', sentence).replace("_","").split()

In [6]:
from tqdm import tqdm
import regex as re
import os

def dump_fasttext(x_data, y_data, file_name):
    docs = list()
    try:
        os.remove(file_name)
    except:
        pass
    with open(file_name, 'a', encoding="utf-8") as file:
        for idx, doc in enumerate(tqdm(map(clean_tokenize, x_data), total=len(x_data))):
            docs.append(doc)
            file.write("__label__{} {}\n".format(y_data[idx], ' '.join(doc)))
    return docs

In [7]:
train_docs = dump_fasttext(x_train_, y_train_, "data.train")
valid_docs = dump_fasttext(x_valid_, y_valid_, "data.valid")

100%|████████████████████████████████████████████████████████████████████████████████████| 319/319 [00:01<00:00, 162.77it/s]


In [8]:
models = fasttext.train_supervised("data.train", epoch=1000, lr=.1, verbose=True)

In [9]:
labels, values = models.predict([" ".join(doc) for doc in x_valid_], num_classes)

In [10]:
values, labels

(array([[2.79210478e-01, 2.58011013e-01, 1.57797664e-01, ...,
         5.26846610e-02, 4.93810400e-02, 4.62223310e-04],
        [9.96729434e-01, 3.26973828e-03, 2.70742439e-05, ...,
         1.00633270e-05, 1.00046009e-05, 1.00000034e-05],
        [9.99552369e-01, 4.62098804e-04, 1.35976170e-05, ...,
         1.02402219e-05, 1.00017296e-05, 1.00000034e-05],
        ...,
        [1.00000012e+00, 1.54307654e-05, 1.25479464e-05, ...,
         1.00002990e-05, 1.00000698e-05, 1.00000034e-05],
        [9.98344123e-01, 1.49778347e-03, 1.82304750e-04, ...,
         1.00000316e-05, 1.00000125e-05, 1.00000034e-05],
        [9.92329419e-01, 7.67517369e-03, 1.91806339e-05, ...,
         1.01073392e-05, 1.00373900e-05, 1.00000034e-05]]),
 [['__label__entertainment',
   '__label__business/finance',
   '__label__education/research',
   '__label__tech/science',
   '__label__politics/government/law',
   '__label__sports',
   '__label__health/medical',
   '__label__news/press'],
  ['__label__sports',
  

In [11]:
from sklearn.metrics import classification_report
print(classification_report(np.array(labels)[:, 0], y_valid_))

C:\Users\luism\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luism\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                             precision    recall  f1-score   support

__label__education/research       0.00      0.00      0.00       7.0
     __label__entertainment       0.00      0.00      0.00       5.0
        __label__news/press       0.00      0.00      0.00       3.0
            __label__sports       0.00      0.00      0.00     304.0
           business/finance       0.00      0.00      0.00       0.0
         education/research       0.00      0.00      0.00       0.0
              entertainment       0.00      0.00      0.00       0.0
             health/medical       0.00      0.00      0.00       0.0
                 news/press       0.00      0.00      0.00       0.0
    politics/government/law       0.00      0.00      0.00       0.0
                     sports       0.00      0.00      0.00       0.0
               tech/science       0.00      0.00      0.00       0.0

                   accuracy                           0.00     319.0
                  macro avg     